In [1]:
import matplotlib.pyplot as plt
import pprint
from pims import ND2Reader_SDK
pp = pprint.PrettyPrinter(indent=0)
from tabulate import tabulate
from tqdm.auto import tqdm 
import cv2
import numpy as np
from cowpy import cow
from celluloid import Camera
import colorama
from termcolor import colored
from joblib import Parallel, delayed, parallel_backend
import yaml
import argparse
import os

In [2]:
def save_image(frame, i, img_format,save_directory,FOV,IMG_CHANNELS,channel, fill_empty=False):
    if (np.sum(frame) == 0) and fill_empty: # Check if a frame is empty, as Nikon inserts empty frames when you have some channels being read ever n frames.
        pass
    else:
        if img_format.lower() in "tiff":
            cv2.imwrite(save_directory + 'xy{}_{}_T{}.tif'.format(str(FOV).zfill(3),IMG_CHANNELS[channel],str(i).zfill(4)), frame, [cv2.IMWRITE_TIFF_COMPRESSION, 1])
        elif img_format.lower() in "png":
            cv2.imwrite(save_directory + 'xy{}_{}_T{}.png'.format(str(FOV).zfill(3),IMG_CHANNELS[channel],str(i).zfill(4)), frame)
        else:
            raise Exception("Invalid format, please choose either TIFF or PNG")


In [3]:
save_directory = "/home/georgeos/Dropbox (Cambridge University)/Nikon Nd2 Backup_Bakshi-Lab/gh464/MI13TnaAGFP_exit_dyn/extracted2/"
directory = "/home/georgeos/Dropbox (Cambridge University)/Nikon Nd2 Backup_Bakshi-Lab/gh464/MI13TnaAGFP_exit_dyn/ND2/MI13TnaAGFP_exit_dyn.nd2"
joblib_workers = 50
save_type = "zarr"
t_stop = 20
fill_empty=False


In [4]:
if save_directory[-1] != "/":
    save_directory = save_directory + "/"
if save_type == "zarr":
    try:
        save_directory = os.path.dirname(save_directory)+".zarr"
        os.mkdir(save_directory)
    except:
        pass
else:
    try:
        os.mkdir(save_directory)
    except:
        pass
    
# Get parameters of the experiment
frames =  ND2Reader_SDK(directory)
metadata_dir = save_directory + "/../"
with open(metadata_dir+'metadata.yml', 'w') as outfile:
    yaml.dump(frames.metadata, outfile, default_flow_style=False)

IMG_HEIGHT = frames.metadata["tile_height"]
IMG_WIDTH = frames.metadata["tile_width"]
IMG_CHANNELS_COUNT = frames.metadata["plane_count"]
SEQUENCES = frames.metadata["sequence_count"]
IMG_CHANNELS = []
for x in tqdm(range(IMG_CHANNELS_COUNT), desc = "Getting experiment info - please wait"):
    IMG_CHANNELS.append(frames.metadata["plane_{}".format(x)]["name"])
frames.close() 
num_FOVs = frames.sizes["m"]
if t_stop:
    num_t = t_stop
else:
    num_t = frames.sizes["t"]
if not t_stop:
    assert int(SEQUENCES / num_FOVs) == num_t, "FOVs ({}) and timepoints ({}) do not match sequences ({}) in the experiment - check your inputs".format(num_FOVs,num_t,SEQUENCES)
print(colored("Experiment parameters (please verify before proceeding with extraction".format(num_FOVs), 'blue', attrs=['bold']))
print(tabulate([
    ['TIMEPOINTS', num_t],
    ['FOVs', num_FOVs],
    ['IMG_HEIGHT', IMG_HEIGHT], 
    ['IMG_WIDTH', IMG_WIDTH],
    ["IMG_CHANNELS_COUNT", IMG_CHANNELS_COUNT],
    ['IMG_CHANNELS', IMG_CHANNELS]], headers=['Parameter', 'Value'], tablefmt='orgtbl'))


if save_type:
    img_format = save_type
else:
    print(colored("Choose image format: (TIFF/PNG) ", 'red', attrs=['bold']))
    img_format = input()


FOVs_list = list(range(num_FOVs))
CHANNELS_list = list(range(IMG_CHANNELS_COUNT))

/home/georgeos/miniconda3/envs/nd2_extractor/lib/python3.10/site-packages/pims/base_frames.py:478: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


Getting experiment info - please wait:   0%|          | 0/3 [00:00<?, ?it/s]

Experiment parameters (please verify before proceeding with extraction
| Parameter          | Value                  |
|--------------------+------------------------|
| TIMEPOINTS         | 20                     |
| FOVs               | 39                     |
| IMG_HEIGHT         | 2304                   |
| IMG_WIDTH          | 2304                   |
| IMG_CHANNELS_COUNT | 3                      |
| IMG_CHANNELS       | ['Green', 'PC', 'Red'] |


In [5]:
zarr_dim = (num_FOVs, num_t, IMG_CHANNELS_COUNT, IMG_HEIGHT, IMG_WIDTH)
chunk_size = (1,1,1,IMG_HEIGHT, IMG_WIDTH)
import zarr
from numcodecs import Blosc

In [6]:
zarr_dtype = {
    16: np.uint16,
    8: np.uint8
}

In [10]:


compressor = Blosc(cname='zstd', clevel=6, shuffle=Blosc.BITSHUFFLE)
zarr_file = zarr.open('data2/example.zarr', mode='w', shape=zarr_dim, chunks=chunk_size, dtype=zarr_dtype[frames.metadata["bitsize_memory"]], compressor = compressor)

In [11]:
def zarr_saver(frame, zarr_file, m, t, c):
    zarr_file[m,t,c] = frame


In [12]:
from copy import deepcopy

In [13]:
CHANNELS_list

[0, 1, 2]

In [14]:
with ND2Reader_SDK(directory) as frames:
    for FOV in tqdm(FOVs_list, desc = "Overall (FOV) progress"):
        for channel in tqdm(CHANNELS_list, desc = "Channel progress in FOV {}".format(FOV), leave = False, position = 1):
            frames.iter_axes = 't'
            try:
                frames.default_coords['c'] = channel
            except:
                pass
            frames.default_coords['m'] = FOV
            Parallel(prefer="threads", n_jobs = joblib_workers)(delayed(zarr_saver)(frames[t], zarr_file, FOV, t, channel) for t in tqdm(range(num_t), desc = "Frame progress in channel {}".format(IMG_CHANNELS[channel]), leave = False, position = 2) )


Overall (FOV) progress:   0%|          | 0/39 [00:00<?, ?it/s]

Channel progress in FOV 0:   0%|          | 0/3 [00:00<?, ?it/s]

Frame progress in channel Green:   0%|          | 0/20 [00:00<?, ?it/s]

Frame progress in channel PC:   0%|          | 0/20 [00:00<?, ?it/s]

Frame progress in channel Red:   0%|          | 0/20 [00:00<?, ?it/s]

...

Frame progress in channel Red:   0%|          | 0/20 [00:00<?, ?it/s]

Channel progress in FOV 38:   0%|          | 0/3 [00:00<?, ?it/s]

Frame progress in channel Green:   0%|          | 0/20 [00:00<?, ?it/s]

Frame progress in channel PC:   0%|          | 0/20 [00:00<?, ?it/s]

Frame progress in channel Red:   0%|          | 0/20 [00:00<?, ?it/s]

In [44]:
range(num_t)

range(0, 20)

In [45]:
frames[0]

OSError: File is closed, unable to read data

In [43]:
i

NameError: name 'i' is not defined